In [4]:
# importing the requests library
import requests
import json
import os
import time

import pandas as pd
import numpy as np
from dotenv import dotenv_values

In [6]:
config = dotenv_values(".env")  
BEARER_TOKEN = config['BEARER_TOKEN']
# path where twitter files will be stored
path = "data/"

In [13]:
# define query parameters 
query = "earthquake -minor, -is:reply -is:retweet" #-from:username / -is:retweet
start_time = "2021-10-17T00:00:00.000Z"
end_time = "2021-10-17T23:59:59.000Z"
max_results = "500"
tweet_fields = "created_at,author_id,geo" # geo produes unreliable results. Kept anyway to simulate dirty data
user_fields = 'username,location' # location is same as geo
file_counter = 0
expansions = 'author_id'

BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAJY%2BNgEAAAAA5SvyA6cq3jifLadZEWMEiYgUj%2FI%3Dci6sbF0x3Ya5EckuWJ6L7M4RJhptE16APFL9PMnhZPZXulQg0I'

# put query parameters in a list
query_params = {'query': query,'tweet.fields': tweet_fields, 'user.fields': user_fields,  \
                'start_time': start_time, 'end_time': end_time, 'max_results': max_results,\
                'expansions': expansions}

url = "https://api.twitter.com/2/tweets/search/all"

In [14]:
# define headers for authorization
headers = {"Authorization": "Bearer " + BEARER_TOKEN}

print("Starting to fetch data")

lst_tweets = []

while True:
    # get results according to url and query
    response = None
    response = requests.request("GET", url, headers=headers, params=query_params)
              
    if response.status_code != 200:
         raise Exception(response.status_code, response.text)
    
    
    # combine data to one
    json_response = response.json()
    if 'data' in json_response:
        lst_tweets = lst_tweets + json_response['data']

    break

    # check if more data available, if yes continue process
    file_counter += 1
    if 'meta' in json_response:
        if 'next_token' in json_response['meta']:
            query_params['next_token'] = json_response['meta']['next_token']
            next_token = json_response['meta']['next_token']
            print("Fetching next few tweets, next_token: ",query_params['next_token'])
            time.sleep(4)
        else:
            file_counter = 0
            del query_params['next_token']
            break
    else:
        file_counter = 0
        del query_params['next_token']
        break

print("DONE")

Starting to fetch data
DONE


In [3]:
import glob
import os

# create list of all file names with path
file_list = glob.glob(os.path.join(os.getcwd(), "Data\\twitter", "*.txt"))

tweets = []
for file in file_list:
    with open(file) as f:
        tweets.append(json.load(f))
        
# create df out of the data
df_tweets = pd.DataFrame()
df_users = pd.DataFrame()
for tweet in tweets:
    df_tweets=df_tweets.append(pd.json_normalize(tweet['data']))
    df_users=df_users.append(pd.json_normalize(tweet['includes']['users']))
    

df_users = df_users.rename(columns={'id':'author_id'})
df = pd.merge(df_tweets, df_users, on='author_id').drop_duplicates(subset='id').reset_index(drop=True)



In [29]:
df

,created_at,text,author_id,id,geo.place_id,geo.coordinates.type,geo.coordinates.coordinates,withheld.copyright,withheld.country_codes_x,username,name,location,withheld.country_codes_y
0,2021-10-17T23:59:54.000Z,"#Earthquake M4.7 CANARY ISLANDS, SPAIN REGION ...",809537352909656064,1449887926985363456,NaN,NaN,NaN,NaN,NaN,EQAlerts,Earthquake Monitor,NaN,NaN
1,2021-10-17T23:58:02.000Z,Moderate magnitude 4.3 #earthquake 24 km north...,809537352909656064,1449887457193963525,NaN,NaN,NaN,NaN,NaN,EQAlerts,Earthquake Monitor,NaN,NaN
2,2021-10-17T23:35:21.000Z,#Earthquake M4.4 Mexico: 82 Km Al Oeste De Chi...,809537352909656064,1449881750532939776,NaN,NaN,NaN,NaN,NaN,EQAlerts,Earthquake Monitor,NaN,NaN
3,2021-10-17T23:12:10.000Z,"#Earthquake M4.0 MINAHASA, SULAWESI, INDONESIA...",809537352909656064,1449875917761884164,NaN,NaN,NaN,NaN,NaN,EQAlerts,Earthquake Monitor,NaN,NaN
4,2021-10-17T23:02:48.000Z,"#Earthquake M3.6 ISLAND OF HAWAII, HAWAII 4min...",809537352909656064,1449873557748596739,NaN,NaN,NaN,NaN,NaN,EQAlerts,Earthquake Monitor,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4953,2021-10-17T00:11:48.000Z,"Earthquake kills 3 in Bali, Indonesia, destroy...",2282930023,1449528535132119043,NaN,NaN,NaN,NaN,NaN,7SealsOfTheEnd,7 Seals Of The End,News & Bible verses USA,NaN
4954,2021-10-17T00:08:42.000Z,4.8 Magnitude Earthquake Strikes Indonesia’s B...,367694191,1449527755733753857,NaN,NaN,NaN,NaN,NaN,ANC_News2,Sanjeev Dev Malik,Gurgaon,NaN
4955,2021-10-17T00:08:00.000Z,"Moderate earthquake rocks Bali, killing at lea...",1313407652202975232,1449527580844077059,NaN,NaN,NaN,NaN,NaN,LindaMi14118735,Linda Miranda,NaN,NaN
4956,2021-10-17T00:05:38.000Z,"A terrible #earthquake hits #Indonesia, causin...",757114028489580544,1449526984086855681,NaN,NaN,NaN,NaN,NaN,News_Disaster1,نُذر العذاب nibiru,منتديات البشرى الإسلاميه,NaN


In [4]:
# create df with usernames and number of postings
account_df = df.username.value_counts()
lst_a = account_df.index.tolist()
lst_v = account_df.tolist()

df_a = pd.DataFrame(lst_a).rename(columns={0: 'username'})
df_a['value'] = lst_v

# filter out accounts with more or equal to 175 postings
df_a = df_a.loc[df_a['value'] > 10].sort_values(by=['value'])
lst_bots = df_a.username.tolist()

# add -from: operator to each entry in list 
append_str = '-from:'
lst_filter = [append_str + sub for sub in lst_bots]

# convert list to a string suitable for the query
filter_names_query = ' '.join(lst_filter)

In [5]:
filter_names_query

'-from:quakeupdates -from:jojo2727 -from:MonitorSismico -from:MyComicalLife -from:news_sokuho_bot -from:DiariosRobot -from:EN_NERV -from:GDACS -from:earthquake_jp -from:EQAlerts -from:j1_quake -from:iSachinSrivstva -from:VolcanoEWS -from:ChileAlertaApp -from:earthb0t -from:sexy_vegetables -from:zishin3255 -from:everyEarthquake -from:MapQuake -from:swap_bot_bash -from:eq_map -from:eq_map_es -from:eq_map_ww -from:SEISMOinfo -from:VegaBajaWx -from:WatchOurCity -from:Keith_Event -from:SismoDetector -from:cvb_223 -from:ExBulletinUk -from:EMSC -from:StoixeioJewelry -from:megamodo -from:earthquakevt -from:QuakeBotter -from:twtaka_jp -from:EarthquakeTw -from:ENSO1998 -from:eq_map_ww2 -from:eq_map_es2'

In [35]:
# define query parameters 
query = "earthquake -minor, -is:reply -is:retweet " + filter_names_query
start_time = "2020-12-01T00:00:00.000Z"
end_time = "2020-12-31T23:59:59.000Z"
max_results = "500"
tweet_fields = "created_at,author_id,geo" # geo produes unreliable results. Kept anyway to simulate dirty data
user_fields = 'username,location' # location is same as geo
place_country = 'JP'
file_counter = 0
expansions = 'author_id'



# put query parameters in a list
query_params = {'query': query,'tweet.fields': tweet_fields, 'user.fields': user_fields,  \
                'start_time': start_time, 'end_time': end_time, 'max_results': max_results,\
                'expansions': expansions}

url = "https://api.twitter.com/2/tweets/search/all"

In [ ]:
# define headers for authorization
headers = {"Authorization": "Bearer " + BEARER_TOKEN}

print("Starting to fetch data")


while True:
    # get results according to url and query
    response = None
    response = requests.request("GET", url, headers=headers, params=query_params)
              
    if response.status_code != 200:
         raise Exception(response.status_code, response.text)
    
    # create json out of result
    json_response = response.json()
    
    # write data into txt
    with open(path + "twitter_file_" + str(file_counter) + ".txt",
              "w") as outfile:
        outfile.write(json.dumps(json_response, indent=4))

    # check if more data available, if yes continue process
    file_counter += 1
    if 'meta' in json_response:
        if 'next_token' in json_response['meta']:
            query_params['next_token'] = json_response['meta']['next_token']
            next_token = json_response['meta']['next_token']
            print("Fetching next few tweets, next_token: ",query_params['next_token'])
            time.sleep(4)
        else:
            file_counter = 0
            break
    else:
        file_counter = 0
        break

print("DONE")